In [1]:
import numpy, math, time
from JokeFunc import LinSys
from importlib import reload
LinSys = reload(LinSys)

# Find linear system (Ax = B) of 2D problem

In [2]:
#Problem function F(u)=f(x,y)
def f(x,y):
    return 4*(math.pi)*(math.pi)*math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)-math.sin(2*math.pi*x)*(2-12*y+12*y*y)

#Boundary Conditions
def bf(x,y):
    return 0

#Setup variables
nx = 70 #number_intervals for x
ax = 0
bx = 1
hx = (bx-ax)/nx
hx2 = hx*hx

ny = 70 #number_intervals for y
ay = 0
by = 1
hy = (by-ay)/ny
hy2 = hy*hy

'''approximated coefficient 
[[(c_i-1,j-1),(c_i-1,j),(c_i-1,j+1)]
 [ (c_i,j-1) , (c_i,j) , (c_i,j+1) ]
 [(c_i+1,j-1),(c_i+1,j),(c_i+1,j+1)]]
'''
C = [[   0   , -1/hx2     ,   0   ],
     [ -1/hy2, 2/hx2+2/hy2, -1/hy2],
     [   0   , -1/hx2     ,   0   ]]

A = numpy.zeros(((nx-1)*(ny-1),(nx-1)*(ny-1)))
B = numpy.zeros((nx-1)*(ny-1))
for i in range(nx-1):
    for j in range(ny-1):    
        T = numpy.zeros((nx+1,ny+1))
        T[i:i+3,j:j+3] = C
        Aij = T[1:-1,1:-1]
        A[i*(ny-1)+j] = Aij.reshape(Aij.size)
        s = 0
        if i*j==0 or i==nx-2 or j==ny-2:
            BC = numpy.array([[bf(ax+(i+0)*hx,ay+(j+0)*hy),bf(ax+(i+0)*hx,ay+(j+1)*hy),bf(ax+(i+0)*hx,ay+(j+2)*hy)],
                              [bf(ax+(i+1)*hx,ay+(j+0)*hy),            0              ,bf(ax+(i+1)*hx,ay+(j+2)*hy)],
                              [bf(ax+(i+2)*hx,ay+(j+0)*hy),bf(ax+(i+2)*hx,ay+(j+1)*hy),bf(ax+(i+2)*hx,ay+(j+2)*hy)]])
            T[1:-1,1:-1] = numpy.zeros_like(T[1:-1,1:-1])
            s = numpy.sum(BC*T[i:i+3,j:j+3])
        
        B[i*(ny-1)+j] = f(ax+(i+1)*hx,ay+(j+1)*hy) - s

# Compare solution
##### default variable

Imax = 10000

x0 = zero vector

eps = 1e-3

weight = 1

## 1. Exact solution

In [3]:
B
x1 = numpy.zeros_like(B)
x1.shape

(4761,)

In [4]:
e = math.e
def ef(x,y):
    return math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)
x1 = numpy.zeros_like(B)
for i in range(nx-1):
    for j in range(ny-1):
        x1[i*(ny-1)+j] = ef(ax+hx*(i+1),ay+hy*(j+1))

## 2. Solution from x = inv(A)*b

In [5]:
t = time.time()
x2 = (numpy.linalg.inv(A)).dot(B)
print(time.time()-t)

1.7672779560089111


## 3.Solution by weighted Jacobi iteration

In [6]:
t = time.time()
x3 = LinSys.wJacobi(A,B)
print(time.time()-t)

1 106.72707914645724
2 106.08367595265521
3 105.5175281120337
4 104.99674093767747
5 104.5069853471073
6 104.04034770666723
7 103.59190217997872
8 103.15833078707769
9 102.73726915776453
10 102.32695971307066
11 101.92605216662331
12 101.53348135132521
13 101.14838859077537
14 100.77006900876486
15 100.39793501563625
16 100.03149027801275
17 99.67031070668722
18 99.31403027590395
19 98.96233025030455
20 98.61493086704921
21 98.27158482046507
22 97.93207209244446
23 97.59619580279009
24 97.26377884313825
25 96.93466112031672
26 96.60869727903648
27 96.2857548054726
28 95.96571243636984
29 95.64845881535472
30 95.33389135087874
31 95.02191523984563
32 94.71244262833027
33 94.40539188646349
34 94.10068697896703
35 93.7982569162769
36 93.49803527392865
37 93.19995977004952
38 92.90397189254755
39 92.61001656899325
40 92.31804187332989
41 92.02799876448114
42 91.73984085268835
43 91.453524190041
44 91.16900708218624
45 90.88624991863963
46 90.60521501948237
47 90.32586649653767
48 90.048170

387 36.40494933767747
388 36.31239616119617
389 36.220085858812745
390 36.12801771396967
391 36.03619101303193
392 35.94460504526755
393 35.85325910282903
394 35.7621524807335
395 35.67128447684443
396 35.58065439185213
397 35.49026152925619
398 35.40010519534637
399 35.31018469918496
400 35.220499352588085
401 35.13104847010845
402 35.04183136901726
403 34.95284736928737
404 34.8640957935747
405 34.775575967202485
406 34.68728721814309
407 34.5992288770018
408 34.51140027699959
409 34.423800753957146
410 34.33642964627806
411 34.24928629493225
412 34.16237004344086
413 34.07568023785927
414 33.98921622676147
415 33.9029773612251
416 33.81696299481493
417 33.7311724835681
418 33.64560518597858
419 33.56026046298242
420 33.47513767794183
421 33.390236196631626
422 33.30555538722367
423 33.2210946202722
424 33.13685326869985
425 33.05283070778353
426 32.96902631513924
427 32.8854394707086
428 32.80206955674529
429 32.71891595780056
430 32.635978060709796
431 32.553255254578936
432 32.470

## 4.Solution by SOR

In [7]:
t = time.time()
x4 = LinSys.SOR(A,B,weight=1)
print(time.time()-t)

max time ~ 13.826560020446777
SOR solution dosen't change
7.887998580932617


## 5.Solution by gradiant method

In [8]:
t = time.time()
x5 = LinSys.GM(A,B)
print(time.time()-t)

max time ~ 27.751370191574097
solution dosen't change
26.233318567276


## 6.Solution by conjugate gradiant method

In [9]:
t = time.time()
x6 = LinSys.CG(A,B,eps=1e-3)
print(time.time()-t)

solution converges
0.27103304862976074


# Compare solution with exact solution

In [10]:
print('Inverse \t\t',numpy.linalg.norm(x1-x2))
print('weighted Jacobi \t',numpy.linalg.norm(x1-x3))
print('SOR \t\t\t',numpy.linalg.norm(x1-x4))
print('gradiant method \t',numpy.linalg.norm(x1-x5))
print('conjugate gradiant \t',numpy.linalg.norm(x1-x6))

Inverse 		 0.0014161180076455668
weighted Jacobi 	 0.39448233699992513
SOR 			 0.197043669707906
gradiant method 	 0.7886798296989036
conjugate gradiant 	 0.0014161179946345997
